In [1]:
import os
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ✅ 한글 폰트 설정 (Windows & Mac 대응)
plt.rcParams["font.family"] = "Malgun Gothic"  # Windows (맑은 고딕)
plt.rcParams["axes.unicode_minus"] = False  # 마이너스 기호 깨짐 방지

# ✅ SQLite 데이터베이스 연결
db_path = "asia (1).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ `career` 컬럼 내용만 삭제
cursor.execute("UPDATE duty_analysis SET career = NULL;")
conn.commit()
print("🔄 기존 `career` 컬럼 데이터 초기화 완료.")

# ✅ 이미지 저장 폴더 생성
output_dir = "career_charts"
os.makedirs(output_dir, exist_ok=True)

# ✅ SQL 쿼리 실행: 직무별 경력 요구 비율 조회
sql_query = """
WITH career_data AS (
    SELECT 'PM' AS category, career FROM processing WHERE duty LIKE '%PM%'
    UNION ALL
    SELECT '데이터 직무' AS category, career FROM processing WHERE duty LIKE '%데이터 직무%'
    UNION ALL
    SELECT '백엔드' AS category, career FROM processing WHERE duty LIKE '%백엔드%'
    UNION ALL
    SELECT '인프라 엔지니어' AS category, career FROM processing WHERE duty LIKE '%인프라 엔지니어%'
    UNION ALL
    SELECT '앱 개발자' AS category, career FROM processing WHERE duty LIKE '%앱 개발자%'
    UNION ALL
    SELECT '게임' AS category, career FROM processing WHERE duty LIKE '%게임%'
    UNION ALL
    SELECT 'AI' AS category, career FROM processing WHERE duty LIKE '%AI%'
    UNION ALL
    SELECT '임베디드' AS category, career FROM processing WHERE duty LIKE '%임베디드%'
    UNION ALL
    SELECT '프론트 엔드' AS category, career FROM processing WHERE duty LIKE '%프론트 엔드%'
    UNION ALL
    SELECT 'QA' AS category, career FROM processing WHERE duty LIKE '%QA%'
    UNION ALL
    SELECT '데이터 분석' AS category, career FROM processing WHERE duty LIKE '%데이터 분석%'
    UNION ALL
    SELECT 'VR' AS category, career FROM processing WHERE duty LIKE '%VR%'
    UNION ALL
    SELECT '시스템' AS category, career FROM processing WHERE duty LIKE '%시스템%'
    UNION ALL
    SELECT '블록체인' AS category, career FROM processing WHERE duty LIKE '%블록체인%'
    UNION ALL
    SELECT 'ERP' AS category, career FROM processing WHERE duty LIKE '%ERP%'
    UNION ALL
    SELECT '언어별 개발자' AS category, career FROM processing WHERE duty LIKE '%언어별 개발자%'
)

SELECT
    category AS 직무,
    career AS 연차,
    COUNT(*) AS 채용_공고_개수
FROM career_data
GROUP BY category, career
ORDER BY category, career;
"""

# ✅ 데이터 로드
df = pd.read_sql_query(sql_query, conn)
df.columns = df.columns.str.strip()  # 컬럼명 공백 제거
cursor.execute(sql_query)
rows = cursor.fetchall()
print(rows)

# ✅ 연차 컬럼 변환: 숫자값 + "년차" 형식 적용
df["연차"] = df["연차"].astype(str) + "년차"

# ✅ 직무별 시각화 및 이미지 저장
unique_jobs = df["직무"].unique()

for job in unique_jobs:
    df_job = df[df["직무"] == job]  # 특정 직무 데이터 필터링
    colors = sns.color_palette("husl", len(df_job["연차"].unique()))  # 연차별 색상 지정

    plt.figure(figsize=(10, 6))
    sns.barplot(data=df_job, x="연차", y="채용_공고_개수", hue="연차", palette=colors, dodge=False, legend=False)

    # ✅ 그래프 꾸미기
    plt.xlabel("경력 연차")
    plt.ylabel("채용 공고 개수")
    plt.title(f"{job} 직무의 연차별 채용 공고 개수")
    plt.xticks(rotation=0)  # 연차 숫자 정자로 정렬
    plt.grid(axis="y", linestyle="--", alpha=0.7)

    # ✅ 이미지 저장 (파일명: "{직무명}.png")
    image_path = os.path.join(output_dir, f"{job}.png")
    plt.savefig(image_path, bbox_inches="tight")
    plt.close()

    # ✅ `career` 컬럼에 이미지 경로만 저장
    cursor.execute(
        "UPDATE duty_analysis SET career = ? WHERE duty = ?;",
        (image_path, job)
    )

# ✅ 데이터베이스 업데이트 반영
conn.commit()
print("✅ 모든 직무별 시각화 이미지가 저장되고 `duty_analysis` 테이블에 반영되었습니다.")

# ✅ DB 연결 종료
conn.close()


🔄 기존 `career` 컬럼 데이터 초기화 완료.
[('AI', 0, 90), ('AI', 1, 45), ('AI', 2, 75), ('AI', 3, 132), ('AI', 4, 12), ('AI', 5, 63), ('AI', 6, 8), ('AI', 7, 13), ('AI', 8, 4), ('AI', 10, 7), ('ERP', 0, 1), ('ERP', 1, 3), ('ERP', 2, 4), ('ERP', 3, 11), ('ERP', 4, 8), ('ERP', 5, 6), ('ERP', 7, 4), ('PM', 0, 12), ('PM', 1, 12), ('PM', 2, 11), ('PM', 3, 36), ('PM', 4, 3), ('PM', 5, 48), ('PM', 6, 5), ('PM', 7, 11), ('PM', 8, 7), ('PM', 10, 7), ('QA', 0, 26), ('QA', 1, 19), ('QA', 2, 20), ('QA', 3, 49), ('QA', 4, 10), ('QA', 5, 25), ('QA', 6, 3), ('QA', 7, 7), ('QA', 8, 1), ('QA', 10, 2), ('VR', 0, 3), ('VR', 2, 3), ('VR', 3, 5), ('VR', 4, 1), ('VR', 5, 3), ('게임', 0, 3), ('게임', 2, 3), ('게임', 3, 10), ('게임', 5, 7), ('데이터 분석', 0, 27), ('데이터 분석', 1, 14), ('데이터 분석', 2, 22), ('데이터 분석', 3, 38), ('데이터 분석', 4, 2), ('데이터 분석', 5, 17), ('데이터 분석', 6, 1), ('데이터 분석', 7, 3), ('데이터 분석', 8, 1), ('데이터 분석', 10, 7), ('데이터 직무', 0, 53), ('데이터 직무', 1, 22), ('데이터 직무', 2, 49), ('데이터 직무', 3, 101), ('데이터 직무', 4, 21), ('데이터 직무', 5

In [2]:
import os
import sqlite3
import openai

# ✅ OpenAI API 키 설정
openai.api_key = ""

# ✅ SQLite 데이터베이스 연결
db_path = "asia (1).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [ ]:
import os
import sqlite3
import openai

# ✅ OpenAI API 키 설정
openai.api_key = ""

# ✅ SQLite 데이터베이스 연결
db_path = "asia (1).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ SQL 쿼리 실행: 직무별 경력 요구 비율 조회
sql_query = """
WITH career_data AS (
    SELECT 'PM' AS category, career FROM processing WHERE duty LIKE '%PM%'
    UNION ALL
    SELECT '데이터 직무' AS category, career FROM processing WHERE duty LIKE '%데이터 직무%'
    UNION ALL
    SELECT '백엔드' AS category, career FROM processing WHERE duty LIKE '%백엔드%'
    UNION ALL
    SELECT '인프라 엔지니어' AS category, career FROM processing WHERE duty LIKE '%인프라 엔지니어%'
    UNION ALL
    SELECT '앱 개발자' AS category, career FROM processing WHERE duty LIKE '%앱 개발자%'
    UNION ALL
    SELECT '게임' AS category, career FROM processing WHERE duty LIKE '%게임%'
    UNION ALL
    SELECT 'AI' AS category, career FROM processing WHERE duty LIKE '%AI%'
    UNION ALL
    SELECT '임베디드' AS category, career FROM processing WHERE duty LIKE '%임베디드%'
    UNION ALL
    SELECT '프론트 엔드' AS category, career FROM processing WHERE duty LIKE '%프론트 엔드%'
    UNION ALL
    SELECT 'QA' AS category, career FROM processing WHERE duty LIKE '%QA%'
    UNION ALL
    SELECT '데이터 분석' AS category, career FROM processing WHERE duty LIKE '%데이터 분석%'
    UNION ALL
    SELECT 'VR' AS category, career FROM processing WHERE duty LIKE '%VR%'
    UNION ALL
    SELECT '시스템' AS category, career FROM processing WHERE duty LIKE '%시스템%'
    UNION ALL
    SELECT '블록체인' AS category, career FROM processing WHERE duty LIKE '%블록체인%'
    UNION ALL
    SELECT 'ERP' AS category, career FROM processing WHERE duty LIKE '%ERP%'
    UNION ALL
    SELECT '언어별 개발자' AS category, career FROM processing WHERE duty LIKE '%언어별 개발자%'
)

SELECT
    category AS 직무,
    career AS 연차,
    COUNT(*) AS 채용_공고_개수
FROM career_data
GROUP BY category, career
ORDER BY category, career;
"""
# ✅ 데이터 가져오기
cursor.execute(sql_query)
rows = cursor.fetchall()
print(rows)

In [4]:
import os
import sqlite3
import openai

# ✅ OpenAI API 키 설정
openai.api_key = ""

# ✅ SQLite 데이터베이스 연결
db_path = "asia (1).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ SQL 쿼리 실행: 직무별 경력 요구 비율 조회
sql_query = """
WITH career_data AS (
    SELECT 'PM' AS category, career FROM processing WHERE duty LIKE '%PM%'
    UNION ALL
    SELECT '데이터 직무' AS category, career FROM processing WHERE duty LIKE '%데이터 직무%'
    UNION ALL
    SELECT '백엔드' AS category, career FROM processing WHERE duty LIKE '%백엔드%'
    UNION ALL
    SELECT '인프라 엔지니어' AS category, career FROM processing WHERE duty LIKE '%인프라 엔지니어%'
    UNION ALL
    SELECT '앱 개발자' AS category, career FROM processing WHERE duty LIKE '%앱 개발자%'
    UNION ALL
    SELECT '게임' AS category, career FROM processing WHERE duty LIKE '%게임%'
    UNION ALL
    SELECT 'AI' AS category, career FROM processing WHERE duty LIKE '%AI%'
    UNION ALL
    SELECT '임베디드' AS category, career FROM processing WHERE duty LIKE '%임베디드%'
    UNION ALL
    SELECT '프론트 엔드' AS category, career FROM processing WHERE duty LIKE '%프론트 엔드%'
    UNION ALL
    SELECT 'QA' AS category, career FROM processing WHERE duty LIKE '%QA%'
    UNION ALL
    SELECT '데이터 분석' AS category, career FROM processing WHERE duty LIKE '%데이터 분석%'
    UNION ALL
    SELECT 'VR' AS category, career FROM processing WHERE duty LIKE '%VR%'
    UNION ALL
    SELECT '시스템' AS category, career FROM processing WHERE duty LIKE '%시스템%'
    UNION ALL
    SELECT '블록체인' AS category, career FROM processing WHERE duty LIKE '%블록체인%'
    UNION ALL
    SELECT 'ERP' AS category, career FROM processing WHERE duty LIKE '%ERP%'
    UNION ALL
    SELECT '언어별 개발자' AS category, career FROM processing WHERE duty LIKE '%언어별 개발자%'
)

SELECT
    category AS 직무,
    career AS 연차,
    COUNT(*) AS 채용_공고_개수
FROM career_data
GROUP BY category, career
ORDER BY category, career;
"""

# ✅ 데이터 가져오기
cursor.execute(sql_query)
rows = cursor.fetchall()
print(rows)

# ✅ 데이터 변환 (직무별 연차별 채용 공고 개수 정리)
job_career_dict = {}
for duty, career, job_count in rows:
    if duty not in job_career_dict:
        job_career_dict[duty] = []
    job_career_dict[duty].append((career, job_count))

# ✅ GPT-3.5 요약 생성 및 `duty_analysis` 테이블 업데이트
for duty, career_data in job_career_dict.items():
    # ✅ 프롬프트 생성
    job_details = "\n".join([f"- {c[0]}년차: {c[1]}건" for c in career_data])
    prompt = f"""
    "{duty}" 직무의 채용 공고 데이터를 분석해주세요.

    [직무별 연차별 채용 공고 개수]
    {job_details}

    위 데이터를 기반으로 다음을 분석해주세요:
    1. 가장 많이 요청된 경력 연차와 해당 채용 공고 수
    2. 두 번째로 많이 요청된 경력 연차와 해당 채용 공고 수
    3. 최소 요구되는 연차
    4. {duty} 직무의 경력별 채용 특징 및 트렌드 분석

    요약은 간결하고 핵심적인 정보만 포함하여 정리해주세요. (단, 경력 0년차는 신입을 뜻하므로, )
    """

    try:
        # ✅ GPT-3.5 API 호출
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        gpt3_summary = response.choices[0].message.content.strip()

        # ✅ 기존 career 컬럼 값 가져오기 (이미지 경로만 존재)
        cursor.execute("SELECT career FROM duty_analysis WHERE duty = ?;", (duty,))
        existing_career_data = cursor.fetchone()

        if existing_career_data and existing_career_data[0].endswith(".png"):
            image_path = existing_career_data[0]  # 이미지 경로 유지
        else:
            image_path = ""

        # ✅ `career` 값 업데이트 (['이미지 경로', 'GPT-3.5 설명'])
        career_list = [image_path, gpt3_summary] if image_path else [gpt3_summary]
        updated_text = str(career_list)  # 리스트를 문자열로 변환

        cursor.execute(
            "UPDATE duty_analysis SET career = ? WHERE duty = ?;",
            (updated_text, duty)
        )
        conn.commit()

        print(f"✅ {duty} 요약 완료 (GPT-3.5 적용): {gpt3_summary}")

    except Exception as e:
        print(f"❌ {duty} 요약 실패: {e}")

# ✅ DB 연결 종료
conn.close()
print("🎯 모든 직무의 경력 요구 사항 요약이 GPT-3.5로 업데이트되었습니다.")


[('AI', 0, 90), ('AI', 1, 45), ('AI', 2, 75), ('AI', 3, 132), ('AI', 4, 12), ('AI', 5, 63), ('AI', 6, 8), ('AI', 7, 13), ('AI', 8, 4), ('AI', 10, 7), ('ERP', 0, 1), ('ERP', 1, 3), ('ERP', 2, 4), ('ERP', 3, 11), ('ERP', 4, 8), ('ERP', 5, 6), ('ERP', 7, 4), ('PM', 0, 12), ('PM', 1, 12), ('PM', 2, 11), ('PM', 3, 36), ('PM', 4, 3), ('PM', 5, 48), ('PM', 6, 5), ('PM', 7, 11), ('PM', 8, 7), ('PM', 10, 7), ('QA', 0, 26), ('QA', 1, 19), ('QA', 2, 20), ('QA', 3, 49), ('QA', 4, 10), ('QA', 5, 25), ('QA', 6, 3), ('QA', 7, 7), ('QA', 8, 1), ('QA', 10, 2), ('VR', 0, 3), ('VR', 2, 3), ('VR', 3, 5), ('VR', 4, 1), ('VR', 5, 3), ('게임', 0, 3), ('게임', 2, 3), ('게임', 3, 10), ('게임', 5, 7), ('데이터 분석', 0, 27), ('데이터 분석', 1, 14), ('데이터 분석', 2, 22), ('데이터 분석', 3, 38), ('데이터 분석', 4, 2), ('데이터 분석', 5, 17), ('데이터 분석', 6, 1), ('데이터 분석', 7, 3), ('데이터 분석', 8, 1), ('데이터 분석', 10, 7), ('데이터 직무', 0, 53), ('데이터 직무', 1, 22), ('데이터 직무', 2, 49), ('데이터 직무', 3, 101), ('데이터 직무', 4, 21), ('데이터 직무', 5, 74), ('데이터 직무', 6, 6), ('데이

In [5]:
import os
import sqlite3
import openai

# ✅ OpenAI API 키 설정
openai.api_key = ""

# ✅ SQLite 데이터베이스 연결
db_path = "asia (1).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ SQL 쿼리 실행: 직무별 경력 요구 비율 조회
sql_query = """
WITH career_data AS (
    SELECT 'PM' AS category, career FROM processing WHERE duty LIKE '%PM%'
    UNION ALL
    SELECT '데이터 직무' AS category, career FROM processing WHERE duty LIKE '%데이터 직무%'
    UNION ALL
    SELECT '백엔드' AS category, career FROM processing WHERE duty LIKE '%백엔드%'
    UNION ALL
    SELECT '인프라 엔지니어' AS category, career FROM processing WHERE duty LIKE '%인프라 엔지니어%'
    UNION ALL
    SELECT '앱 개발자' AS category, career FROM processing WHERE duty LIKE '%앱 개발자%'
    UNION ALL
    SELECT '게임' AS category, career FROM processing WHERE duty LIKE '%게임%'
    UNION ALL
    SELECT 'AI' AS category, career FROM processing WHERE duty LIKE '%AI%'
    UNION ALL
    SELECT '임베디드' AS category, career FROM processing WHERE duty LIKE '%임베디드%'
    UNION ALL
    SELECT '프론트 엔드' AS category, career FROM processing WHERE duty LIKE '%프론트 엔드%'
    UNION ALL
    SELECT 'QA' AS category, career FROM processing WHERE duty LIKE '%QA%'
    UNION ALL
    SELECT '데이터 분석' AS category, career FROM processing WHERE duty LIKE '%데이터 분석%'
    UNION ALL
    SELECT 'VR' AS category, career FROM processing WHERE duty LIKE '%VR%'
    UNION ALL
    SELECT '시스템' AS category, career FROM processing WHERE duty LIKE '%시스템%'
    UNION ALL
    SELECT '블록체인' AS category, career FROM processing WHERE duty LIKE '%블록체인%'
    UNION ALL
    SELECT 'ERP' AS category, career FROM processing WHERE duty LIKE '%ERP%'
    UNION ALL
    SELECT '언어별 개발자' AS category, career FROM processing WHERE duty LIKE '%언어별 개발자%'
)

SELECT
    category AS 직무,
    career AS 연차,
    COUNT(*) AS 채용_공고_개수
FROM career_data
GROUP BY category, career
ORDER BY category, career;
"""

# ✅ 데이터 가져오기
cursor.execute(sql_query)
rows = cursor.fetchall()
print(rows)

# ✅ 데이터 변환 (직무별 연차별 채용 공고 개수 정리)
job_career_dict = {}
for duty, career, job_count in rows:
    if duty not in job_career_dict:
        job_career_dict[duty] = []
    job_career_dict[duty].append((career, job_count))

# ✅ GPT-4 요약 생성 및 `duty_analysis` 테이블 업데이트
for duty, career_data in job_career_dict.items():
    # ✅ 프롬프트 생성
    job_details = "\n".join([f"- {c[0]}년차: {c[1]}건" for c in career_data])
    prompt = f"""
    "{duty}" 직무의 채용 공고 데이터를 분석해주세요.

    [직무별 연차별 채용 공고 개수]
    {job_details}

    위 데이터를 기반으로 다음을 분석해주세요:
    1. 가장 많이 요청된 경력 연차와 해당 채용 공고 수
    2. 두 번째로 많이 요청된 경력 연차와 해당 채용 공고 수
    3. 최소 요구되는 연차
    4. {duty} 직무에서의 경력별 채용 특징 및 트렌드 분석

    요약은 간결하고 핵심적인 정보만 포함하여 정리해주세요.
    """

    try:
        # ✅ GPT-4 API 호출
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        gpt4_summary = response.choices[0].message.content.strip()

        # ✅ 기존 career 컬럼 값 가져오기 (이미지 경로 유지)
        cursor.execute("SELECT career FROM duty_analysis WHERE duty = ?;", (duty,))
        existing_career_data = cursor.fetchone()

        if existing_career_data and existing_career_data[0].endswith(".png"):
            image_path = existing_career_data[0]  # 이미지 경로 유지
        else:
            image_path = ""

        # ✅ `career` 값 업데이트 (['이미지 경로', 'GPT-4 설명'])
        career_list = [image_path, gpt4_summary] if image_path else [gpt4_summary]
        updated_text = str(career_list)  # 리스트를 문자열로 변환

        cursor.execute(
            "UPDATE duty_analysis SET career = ? WHERE duty = ?;",
            (updated_text, duty)
        )
        conn.commit()

        print(f"✅ {duty} 요약 완료 (GPT-4 적용): {gpt4_summary}")

    except Exception as e:
        print(f"❌ {duty} 요약 실패: {e}")

# ✅ DB 연결 종료
conn.close()
print("🎯 모든 직무의 경력 요구 사항 요약이 GPT-4로 업데이트되었습니다.")


[('AI', 0, 90), ('AI', 1, 45), ('AI', 2, 75), ('AI', 3, 132), ('AI', 4, 12), ('AI', 5, 63), ('AI', 6, 8), ('AI', 7, 13), ('AI', 8, 4), ('AI', 10, 7), ('ERP', 0, 1), ('ERP', 1, 3), ('ERP', 2, 4), ('ERP', 3, 11), ('ERP', 4, 8), ('ERP', 5, 6), ('ERP', 7, 4), ('PM', 0, 12), ('PM', 1, 12), ('PM', 2, 11), ('PM', 3, 36), ('PM', 4, 3), ('PM', 5, 48), ('PM', 6, 5), ('PM', 7, 11), ('PM', 8, 7), ('PM', 10, 7), ('QA', 0, 26), ('QA', 1, 19), ('QA', 2, 20), ('QA', 3, 49), ('QA', 4, 10), ('QA', 5, 25), ('QA', 6, 3), ('QA', 7, 7), ('QA', 8, 1), ('QA', 10, 2), ('VR', 0, 3), ('VR', 2, 3), ('VR', 3, 5), ('VR', 4, 1), ('VR', 5, 3), ('게임', 0, 3), ('게임', 2, 3), ('게임', 3, 10), ('게임', 5, 7), ('데이터 분석', 0, 27), ('데이터 분석', 1, 14), ('데이터 분석', 2, 22), ('데이터 분석', 3, 38), ('데이터 분석', 4, 2), ('데이터 분석', 5, 17), ('데이터 분석', 6, 1), ('데이터 분석', 7, 3), ('데이터 분석', 8, 1), ('데이터 분석', 10, 7), ('데이터 직무', 0, 53), ('데이터 직무', 1, 22), ('데이터 직무', 2, 49), ('데이터 직무', 3, 101), ('데이터 직무', 4, 21), ('데이터 직무', 5, 74), ('데이터 직무', 6, 6), ('데이

In [6]:
import os
import sqlite3
import ast  # 문자열로 저장된 리스트 변환을 위해 필요

# ✅ SQLite 데이터베이스 연결
db_path = "asia (1).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ 현재 career 컬럼 값 가져오기
cursor.execute("SELECT duty, career FROM duty_analysis;")
rows = cursor.fetchall()
print(rows)

# ✅ 데이터 업데이트 (이미지 경로 복구 & 설명글 유지)
for duty, career_data in rows:
    if not career_data:
        continue  # 데이터가 없는 경우 건너뜀

    try:
        # ✅ 기존 데이터 리스트 변환
        career_list = ast.literal_eval(career_data) if career_data.startswith("[") else [career_data]
        
        # ✅ 이미지 경로 복구
        image_path = f"career_charts/{duty}.png"

        # ✅ 기존 리스트에 이미지 경로가 없는 경우 추가
        if len(career_list) == 1:
            career_list.insert(0, image_path)
        elif not career_list[0].endswith(".png"):
            career_list[0] = image_path  # 첫 번째 값이 이미지 경로가 아니면 교체

        # ✅ career 컬럼 업데이트
        updated_text = str(career_list)
        cursor.execute("UPDATE duty_analysis SET career = ? WHERE duty = ?;", (updated_text, duty))
        conn.commit()

        print(f"✅ {duty} 직무 업데이트 완료: {updated_text}")

    except Exception as e:
        print(f"❌ {duty} 직무 업데이트 실패: {e}")

# ✅ DB 연결 종료
conn.close()
print("🎯 모든 직무의 career 컬럼이 정상적으로 업데이트되었습니다.")


[('AI', "['1. 가장 많이 요청된 경력 연차는 3년차로, 해당 채용 공고 수는 132건입니다.\\n2. 두 번째로 많이 요청된 경력 연차는 0년차로, 해당 채용 공고 수는 90건입니다.\\n3. 최소 요구되는 연차는 0년차입니다.\\n4. AI 직무에서의 경력별 채용 특징 및 트렌드 분석:\\n   - 경력이 증가할수록 채용 공고 수는 대체로 감소하는 경향을 보입니다. 이는 AI 분야가 비교적 신규 분야라고 판단되며, 높은 연차의 경력자들보다는 초기 단계의 경력자들을 더욱 선호하고 있음을 알 수 있습니다.\\n   - 하지만, 예외적으로 3년차에서 가장 많은 채용 공고가 발생하였습니다. 이는 AI 분야에서 3년 정도의 경력을 쌓은 이후에도 적합한 능력이 있으면 채용될 가능성이 높다는 것을 의미하며, 해당 시기가 특정한 능력과 지식을 충분히 갖추게 되는 중요한 시점임을 나타낼 수 있습니다.\\n   - 최고 연차인 10년차의 경우, 공고 수가 매우 적은 것으로 보아 AI 직무에서는 연차가 높을수록 채용 가능성이 낮아지는 경향이 있는 것으로 판단됩니다.']"), ('ERP', "['1. 가장 많이 요청된 경력 연차는 3년차로 채용 공고 수는 총 11건입니다.\\n2. 두 번째로 많이 요청된 경력 연차는 4년차로 채용 공고 수는 총 8건입니다.\\n3. 최소 요구되는 연차는 0년차입니다.\\n4. ERP 직무에서의 경력별 채용 특징 및 트렌드를 분석해보면, 주로 3년차 이상의 경력자를 원하는 경향이 있습니다. 특히, 3년차의 공고 수가 가장 많은 것으로 보아 ERP 직무는 일정 수준의 경험이 요구되며, 중급자 위주의 채용이 이루어지는 것으로 해석할 수 있습니다. 그러나 0년차를 포함한 신입도 일부 채용되며, 이는 ERP 직무의 특성상 신규 인력도 필요로 한다는 것을 의미할 수 있습니다. 전반적으로 이 데이터는 ERP 직무에 대한 경험과 지식이 중요하게 평가되는 것을 보여줍니다. \\n\\n요약: ERP 직무 채용 공고 데이터를 분석한 결과, 가장 많이 요청

In [7]:
# 경력도 확실하게 입니다 말투로 통일

In [8]:
import sqlite3
import ast  # 문자열을 리스트로 변환

# ✅ SQLite 데이터베이스 연결
db_path = "asia (1).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ 정리된 16개 직무 경력 분석 설명 (2번째 원소로 추가할 내용)
career_descriptions = {
    "AI": [
        "1. 가장 많이 요청된 경력 연차는 3년차로, 해당 채용 공고 수는 132건입니다.",
        "2. 두 번째로 많이 요청된 경력 연차는 0년차로, 해당 채용 공고 수는 90건입니다.",
        "3. 최소 요구되는 연차는 0년차입니다.",
        "4. AI 직무에서의 경력별 채용 특징 및 트렌드 분석:",
        "   - 경력이 증가할수록 채용 공고 수는 대체로 감소하는 경향이 있습니다. 이는 AI 분야가 비교적 신규 분야라고 판단되며, 높은 연차의 경력자보다는 초기 단계의 경력자를 더욱 선호하고 있음을 알 수 있습니다.",
        "   - 하지만, 예외적으로 3년차에서 가장 많은 채용 공고가 발생하였습니다. 이는 AI 분야에서 3년 정도의 경력을 쌓은 이후에도 적합한 능력이 있으면 채용될 가능성이 높다는 것을 의미하며, 해당 시기가 특정한 능력과 지식을 충분히 갖추게 되는 중요한 시점임을 나타낼 수 있습니다.",
        "   - 최고 연차인 10년차의 경우, 공고 수가 매우 적은 것으로 보아 AI 직무에서는 연차가 높을수록 채용 가능성이 낮아지는 경향이 있는 것으로 판단됩니다."
    ], 
    "ERP": [
        "1. 가장 많이 요청된 경력 연차는 3년차로 채용 공고 수는 총 11건입니다.",
        "2. 두 번째로 많이 요청된 경력 연차는 4년차로 채용 공고 수는 총 8건입니다.",
        "3. 최소 요구되는 연차는 0년차입니다.",
        "4. ERP 직무에서의 경력별 채용 특징 및 트렌드 분석:",
        "   - 주로 3년차 이상의 경력자를 원하는 경향이 있습니다. 특히, 3년차의 공고 수가 가장 많은 것으로 보아 ERP 직무는 일정 수준의 경험이 요구되며, 중급자 위주의 채용이 이루어지는 것으로 해석할 수 있습니다.",
        "   - 그러나 0년차를 포함한 신입도 일부 채용되며, 이는 ERP 직무의 특성상 신규 인력도 필요로 한다는 것을 의미할 수 있습니다.",
        "   - 전반적으로 이 데이터는 ERP 직무에 대한 경험과 지식이 중요하게 평가되는 것을 보여줍니다."
    ],
    "PM": [
        "1. 가장 많이 요청된 경력 연차는 5년차로, 해당 채용 공고 수는 총 48건입니다.",
        "2. 두 번째로 많이 요청된 경력 연차는 3년차로, 해당 채용 공고 수는 총 36건입니다.",
        "3. 최소 요구되는 연차는 0년차입니다.",
        "4. PM 직무에서의 경력별 채용 특징 및 트렌드 분석:",
        "   - 중간 경력(3~5년차)이 가장 많이 요구되는 경향이 있으며, 초급 및 전문가층(10년차 이상)의 수요는 상대적으로 적습니다.",
        "   - 이는 해당 직무에서 실질적인 프로젝트 경험과 노하우가 중요하지만, 너무 많은 경력을 요구하지는 않는다는 것으로 해석될 수 있습니다.",
        "   - 따라서 PM 직무에 지원하는 경우 중간 경력을 가진 지원자가 우대받을 가능성이 큽니다."
    ],
    "QA": [
        "1. 가장 많이 요청된 경력 연차는 3년차로, 해당 채용 공고 수는 총 49건입니다.",
        "2. 두 번째로 많이 요청된 경력 연차는 5년차로, 해당 채용 공고 수는 총 25건입니다.",
        "3. 최소 요구되는 연차는 0년차로, 이 경우의 채용 공고 수는 총 26건입니다.",
        "4. QA 직무에서의 경력별 채용 특징 및 트렌드 분석:",
        "   - 주로 3~5년 정도의 경력이 요구되며, 이후 경력에 따른 요구수는 점차 감소하는 경향이 보입니다.",
        "   - 그러나 경력이 없거나 적은 0~2년차의 경우에도 상당한 수의 채용 공고가 있습니다.",
        "   - 이는 QA 직무에 있어 경험이 중요하지만, 신입이나 경력이 적은 인력에게도 기회를 주려는 경향이 있다는 것을 의미합니다."
    ],
    "VR": [
        "1. 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 5건입니다.",
        "2. 두 번째로 많이 요청된 경력 연차는 0년차, 2년차, 5년차로 같은 수치인 3건입니다.",
        "3. 최소 요구되는 연차는 0년차입니다.",
        "4. VR 직무에서의 경력별 채용 특징 및 트렌드 분석:",
        "   - 경력이 높아질수록 채용 공고 수가 감소하는 경향이 있습니다.",
        "   - 가장 많이 요구된 경력 연차는 3년차로, 공고 수도 가장 많았습니다. 이는 VR 분야에서 중간 정도의 경력을 가진 인력을 선호한다는 특징을 보여줍니다.",
        "   - 또한, 최소 요구되는 연차가 0년차인 것을 보아 VR 분야는 신입 인력을 공급받는 비율도 높습니다.",
        "   - VR 직무는 신입부터 중간 경력자까지 다양한 경력 레벨을 뽑는 경향이 있어 경력이 다양해도 기회가 많아 보입니다."
    ],
    "게임": [
        "1. 가장 많이 요청된 경력 연차는 3년차로, 해당 채용 공고 개수는 10건입니다.",
        "2. 두 번째로 많이 요청된 경력 연차는 5년차로, 해당 채용 공고 개수는 7건입니다.",
        "3. 최소 요구되는 연차는 0년차입니다.",
        "4. 게임 직무에서의 경력별 채용 트렌드 분석:",
        "   - 가장 높은 수요는 중간 경력인 3년차에서 나타나며, 꽤 높은 수요는 또한 상대적으로 높은 경력인 5년차에서 나타났습니다.",
        "   - 최소 요구 경력이 0년차인 것으로 보아, 경력이 없어도 지원 가능한 직무가 있지만, 보다 높은 경력을 갖춘 인력에 대한 수요가 더 높아 보입니다.",
        "   - 이는 게임 분야에서 특정 기술 또는 경험이 중요하게 여겨짐을 반영할 수 있습니다."
    ],    
    "데이터 분석": [
        "1. 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 38건입니다.",
        "2. 두 번째로 많이 요청된 경력 연차는 2년차이며, 해당 채용 공고 수는 22건입니다.",
        "3. 최소 요구되는 연차는 0년차입니다.",
        "4. 데이터 분석 직무에서의 경력별 채용 특징 및 트렌드 분석:",
        "   - 0~3년차의 공고가 가장 많으며, 특히 3년차의 경우 다른 연차에 비해 채용 공고 수가 많아 일정 수준의 경력을 가진 직원을 선호하는 경향이 있습니다.",
        "   - 4년차 이후로는 채용 공고가 급격히 줄어, 고경력자의 채용 빈도가 낮음을 나타냅니다.",
        "   - 특히 10년 이상 경력자의 경우 채용 공고 수가 많지 않아, 데이터 분석 직무는 초기 및 중반 경력자에게 주로 기회가 제공되며, 고경력자는 특정한 경우에만 채용되는 것으로 해석할 수 있습니다."
    ],
    "데이터 직무": [
        "1. 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 101건입니다.",
        "2. 두 번째로 많이 요청된 경력 연차는 5년차이며, 해당 채용 공고 수는 74건입니다.",
        "3. 최소 요구되는 연차는 0년차입니다.",
        "4. 데이터 직무에서의 경력별 채용 특징 및 트렌드 분석:",
        "   - 중간 경력자(3년차와 5년차)를 선호하는 경향이 있으며, 0~2년차의 신입급 채용도 비교적 많습니다.",
        "   - 7년차 이후의 고경력자 채용은 급격히 줄어드는 경향을 보입니다.",
        "   - 데이터 직무는 일정 이상의 경력을 요구하지만, 최신 기술과 트렌드에 대한 이해도를 중요하게 평가하는 특징이 있습니다."
    ],
    "백엔드": [
        "1. 가장 많이 요청된 경력 연차는 3년차이며, 채용 공고 수는 429건입니다.",
        "2. 두 번째로 많이 요청된 경력 연차는 5년차이며, 해당 채용 공고 수는 296건입니다.",
        "3. 최소 요구되는 연차는 0년차입니다.",
        "4. 백엔드 직무에서의 경력별 채용 특징 및 트렌드 분석:",
        "   - 3년차와 5년차 경력이 가장 많이 요구되는 경향이 있으며, 이는 백엔드 직무에 일정 수준의 경험과 기술력이 요구됨을 의미합니다.",
        "   - 0년차 채용 공고도 많은 편으로, 신입도 환영하는 기업이 많습니다.",
        "   - 7년차 이상의 고경력자보다는 초기 및 중반 경력의 인력을 선호하는 추세가 두드러집니다."
    ],
    "블록체인": [
        "1. 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 16건입니다.",
        "2. 두 번째로 많이 요청된 경력 연차는 5년차이며, 해당 채용 공고 수는 15건입니다.",
        "3. 최소 요구되는 연차는 0년차입니다.",
        "4. 블록체인 직무에서의 경력별 채용 특징 및 트렌드 분석:",
        "   - 3년차와 5년차의 채용 수요가 가장 높으며, 1~2년차도 꾸준한 요구를 보이고 있습니다.",
        "   - 4년차 이후부터는 채용 공고 수가 감소하는 경향이 있습니다.",
        "   - 블록체인 기술이 비교적 새로운 분야이므로, 경험 많은 인력을 찾기보다는 기술 적응력이 높은 인력을 선호하는 경향이 있습니다."
    ],
    "시스템": [
        "1. 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 104건입니다.",
        "2. 두 번째로 많이 요청된 경력 연차는 5년차이며, 해당 채용 공고 수는 51건입니다.",
        "3. 최소 요구되는 연차는 0년차입니다.",
        "4. 시스템 직무에서의 경력별 채용 특징 및 트렌드 분석:",
        "   - 중간 경력(3~5년차)에서 집중적으로 채용이 이루어지는 경향이 있습니다.",
        "   - 초급(0~2년차) 및 고급 경력(10년 이상)의 경우 상대적으로 채용 공고가 적습니다.",
        "   - 신입부터 중간 경력까지 다양한 채용이 이루어지는 직무라고 볼 수 있습니다."
    ],
    "앱 개발자": [
        "1. 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 99건입니다.",
        "2. 두 번째로 많이 요청된 경력 연차는 5년차이며, 해당 채용 공고 수는 58건입니다.",
        "3. 최소 요구되는 연차는 0년차입니다.",
        "4. 앱 개발자 직무에서의 경력별 채용 특징 및 트렌드 분석:",
        "   - 경력이 증가할수록 채용 공고 수도 증가하는 경향이 있습니다.",
        "   - 3년차와 5년차가 가장 많이 요구되며, 8년 이상 경력을 요구하는 공고는 매우 적습니다.",
        "   - 중~하위 연차의 개발자에 대한 수요가 높으며, 높은 연차의 경력자를 찾는 기업은 상대적으로 적습니다."
    ],
    "언어별 개발자": [
        "1. 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 344건입니다.",
        "2. 두 번째로 많이 요청된 경력 연차는 5년차이며, 해당 채용 공고 수는 249건입니다.",
        "3. 최소 요구되는 연차는 0년차입니다.",
        "4. 언어별 개발자 직무에서의 경력별 채용 특징 및 트렌드 분석:",
        "   - 3~5년차 경력을 요구하는 채용 공고가 가장 많습니다.",
        "   - 0년차 채용 공고도 비교적 많은 편으로, 개발 분야에 신입으로 진입하려는 사람들에게 기회가 열려 있습니다.",
        "   - 전반적으로 일정 수준의 경험이 필요하지만, 신입도 충분히 채용될 수 있는 기회가 존재하는 직무입니다."
    ],
    "인프라 엔지니어": [
        "1. 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 394건입니다.",
        "2. 두 번째로 많이 요청된 경력 연차는 5년차이며, 해당 채용 공고 수는 264건입니다.",
        "3. 최소 요구되는 연차는 0년차입니다.",
        "4. 인프라 엔지니어 직무에서의 경력별 채용 특징 및 트렌드 분석:",
        "   - 3년차와 5년차 경력자가 가장 많이 요구되며, 9년차 이상의 고경력자 채용 수요는 적습니다.",
        "   - 일정 수준의 기술력을 가진 인력을 채용 후 지속적인 성장과 발전을 기대하는 경향이 있습니다."
    ],
    "임베디드": [
        "1. 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 86건입니다.",
        "2. 두 번째로 많이 요청된 경력 연차는 0년차이며, 해당 채용 공고 수는 63건입니다.",
        "3. 최소 요구되는 연차는 0년차입니다.",
        "4. 임베디드 직무에서의 경력별 채용 특징 및 트렌드 분석:",
        "   - 3년차 이하의 경력을 요구하는 공고가 대부분입니다.",
        "   - 신규 기술 습득과 유연한 기술 적응 능력이 중요한 분야로, 신입 및 초중급 경력자의 채용 비중이 높습니다."
    ],
    "프론트 엔드": [
        "1. 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 300건입니다.",
        "2. 두 번째로 많이 요청된 경력 연차는 5년차이며, 해당 채용 공고 수는 194건입니다.",
        "3. 최소 요구되는 연차는 0년차입니다.",
        "4. 프론트 엔드 직무에서의 경력별 채용 특징 및 트렌드 분석:",
        "   - 3년차 이상의 경력자를 주로 찾는 경향이 있으며, 3~5년차의 채용 기회가 가장 많습니다.",
        "   - 그 이상의 경력자는 상대적으로 채용 공고 수가 줄어드는 특징이 있습니다.",
        "   - 프론트 엔드 직무는 지속적으로 새로운 기술과 트렌드 변화를 따라가야 하는 분야여서 유연한 기술 적응 능력이 중요합니다.",
        "   - 0년차 채용 공고도 상당히 많은 편이며, 이는 신입을 대상으로 한 채용이 활발히 이루어지고 있음을 나타냅니다.",
        "   - 결과적으로, 신선한 아이디어와 유연한 대처 능력을 갖춘 중간 경력자들이 가장 선호되며, 신입과 고경력자에게도 일정한 기회가 제공됩니다."
    ]
}


# ✅ career 컬럼 업데이트
for duty, description in career_descriptions.items():
    try:
        # ✅ 기존 career 값 가져오기
        cursor.execute("SELECT career FROM duty_analysis WHERE duty = ?;", (duty,))
        existing_data = cursor.fetchone()

        if existing_data and existing_data[0]:
            career_list = ast.literal_eval(existing_data[0])  # 문자열을 리스트로 변환
        else:
            career_list = [f"career_charts/{duty}.png", ""]  # 기본값 설정

        # ✅ 첫 번째 원소(이미지 경로) 유지하고 두 번째 원소 업데이트
        career_list[1] = " ".join(description)

        # ✅ 업데이트된 데이터 저장
        updated_text = str(career_list)
        cursor.execute("UPDATE duty_analysis SET career = ? WHERE duty = ?;", (updated_text, duty))
        conn.commit()

        print(f"{duty} 직무 career 컬럼 업데이트 완료: {updated_text}")

    except Exception as e:
        print(f"{duty} 직무 career 컬럼 업데이트 실패: {e}")

# ✅ DB 연결 종료
conn.close()
print("🎯 모든 직무의 career 컬럼 업데이트 완료되었습니다.")


AI 직무 career 컬럼 업데이트 완료: ['career_charts/AI.png', '1. 가장 많이 요청된 경력 연차는 3년차로, 해당 채용 공고 수는 132건입니다. 2. 두 번째로 많이 요청된 경력 연차는 0년차로, 해당 채용 공고 수는 90건입니다. 3. 최소 요구되는 연차는 0년차입니다. 4. AI 직무에서의 경력별 채용 특징 및 트렌드 분석:    - 경력이 증가할수록 채용 공고 수는 대체로 감소하는 경향이 있습니다. 이는 AI 분야가 비교적 신규 분야라고 판단되며, 높은 연차의 경력자보다는 초기 단계의 경력자를 더욱 선호하고 있음을 알 수 있습니다.    - 하지만, 예외적으로 3년차에서 가장 많은 채용 공고가 발생하였습니다. 이는 AI 분야에서 3년 정도의 경력을 쌓은 이후에도 적합한 능력이 있으면 채용될 가능성이 높다는 것을 의미하며, 해당 시기가 특정한 능력과 지식을 충분히 갖추게 되는 중요한 시점임을 나타낼 수 있습니다.    - 최고 연차인 10년차의 경우, 공고 수가 매우 적은 것으로 보아 AI 직무에서는 연차가 높을수록 채용 가능성이 낮아지는 경향이 있는 것으로 판단됩니다.']
ERP 직무 career 컬럼 업데이트 완료: ['career_charts/ERP.png', '1. 가장 많이 요청된 경력 연차는 3년차로 채용 공고 수는 총 11건입니다. 2. 두 번째로 많이 요청된 경력 연차는 4년차로 채용 공고 수는 총 8건입니다. 3. 최소 요구되는 연차는 0년차입니다. 4. ERP 직무에서의 경력별 채용 특징 및 트렌드 분석:    - 주로 3년차 이상의 경력자를 원하는 경향이 있습니다. 특히, 3년차의 공고 수가 가장 많은 것으로 보아 ERP 직무는 일정 수준의 경험이 요구되며, 중급자 위주의 채용이 이루어지는 것으로 해석할 수 있습니다.    - 그러나 0년차를 포함한 신입도 일부 채용되며, 이는 ERP 직무의 특성상 신규 인력도 필요로 한다는 것을 의미할 수 있습니다.    - 전반적으로 이 데이터는 ERP 직무에 대한 경험

In [ ]:
# 2월 6일 멘토님 피드백 
# 경력 0년차는 신입을 뜻하므로, 내용이 어색하다. 바꿀 것 

In [1]:
import sqlite3
import ast  # 문자열 리스트 변환용

# ✅ SQLite 데이터베이스 연결
db_path = "asia (2).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ `duty_analysis` 테이블의 career 컬럼 조회
cursor.execute("SELECT duty, career FROM duty_analysis;")
rows = cursor.fetchall()

# ✅ 2번째 원소 값 추출
for duty, career in rows:
    if career:
        try:
            # 문자열로 저장된 리스트를 실제 리스트로 변환
            career_list = ast.literal_eval(career)

            # 리스트가 있고, 최소 2개의 원소가 있을 경우 출력
            if isinstance(career_list, list) and len(career_list) > 1:
                print(f"📌 {duty}: {career_list[1]}")
            else:
                print(f"⚠️ {duty}: career 데이터가 부족합니다.")
        except Exception as e:
            print(f"❌ {duty}: 데이터 변환 오류 - {e}")

# ✅ DB 연결 종료
conn.close()


📌 AI: 1. 가장 많이 요청된 경력 연차는 3년차로, 해당 채용 공고 수는 132건입니다. 2. 두 번째로 많이 요청된 경력 연차는 0년차로, 해당 채용 공고 수는 90건입니다. 3. 최소 요구되는 연차는 0년차입니다. 4. AI 직무에서의 경력별 채용 특징 및 트렌드 분석:    - 경력이 증가할수록 채용 공고 수는 대체로 감소하는 경향이 있습니다. 이는 AI 분야가 비교적 신규 분야라고 판단되며, 높은 연차의 경력자보다는 초기 단계의 경력자를 더욱 선호하고 있음을 알 수 있습니다.    - 하지만, 예외적으로 3년차에서 가장 많은 채용 공고가 발생하였습니다. 이는 AI 분야에서 3년 정도의 경력을 쌓은 이후에도 적합한 능력이 있으면 채용될 가능성이 높다는 것을 의미하며, 해당 시기가 특정한 능력과 지식을 충분히 갖추게 되는 중요한 시점임을 나타낼 수 있습니다.    - 최고 연차인 10년차의 경우, 공고 수가 매우 적은 것으로 보아 AI 직무에서는 연차가 높을수록 채용 가능성이 낮아지는 경향이 있는 것으로 판단됩니다.
📌 ERP: 1. 가장 많이 요청된 경력 연차는 3년차로 채용 공고 수는 총 11건입니다. 2. 두 번째로 많이 요청된 경력 연차는 4년차로 채용 공고 수는 총 8건입니다. 3. 최소 요구되는 연차는 0년차입니다. 4. ERP 직무에서의 경력별 채용 특징 및 트렌드 분석:    - 주로 3년차 이상의 경력자를 원하는 경향이 있습니다. 특히, 3년차의 공고 수가 가장 많은 것으로 보아 ERP 직무는 일정 수준의 경험이 요구되며, 중급자 위주의 채용이 이루어지는 것으로 해석할 수 있습니다.    - 그러나 0년차를 포함한 신입도 일부 채용되며, 이는 ERP 직무의 특성상 신규 인력도 필요로 한다는 것을 의미할 수 있습니다.    - 전반적으로 이 데이터는 ERP 직무에 대한 경험과 지식이 중요하게 평가되는 것을 보여줍니다.
📌 PM: 1. 가장 많이 요청된 경력 연차는 5년차로, 해당 채용 공고 수는 총 48건입니다. 2. 두 번째로 많이 요

In [5]:
import sqlite3
import ast  # 문자열을 리스트로 변환하는 데 사용

# ✅ SQLite 데이터베이스 연결
db_path = "asia (2).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ `duty_analysis` 테이블에서 duty와 career 조회
cursor.execute("SELECT duty, career FROM duty_analysis;")
rows = cursor.fetchall()

# ✅ 각 직무별 career의 두 번째 원소 출력
for duty, career in rows:
    try:
        # career 컬럼이 리스트 형식으로 저장된 문자열이라면 변환
        career_list = ast.literal_eval(career) if career else []
        
        # 리스트가 정상적으로 변환되었고 두 번째 원소가 존재하면 출력
        if isinstance(career_list, list) and len(career_list) > 1:
            print(f"{duty}:\n{career_list[1]}\n")
        else:
            print(f"{duty}: 데이터 없음\n")

    except (SyntaxError, ValueError):
        print(f"{duty}: 데이터 변환 오류\n")

# ✅ DB 연결 종료
conn.close()


AI:
1. 가장 많이 요청된 경력 연차는 3년차로, 해당 채용 공고 수는 132건입니다. 2. 두 번째로 많이 요청된 경력 연차는 0년차로, 해당 채용 공고 수는 90건입니다. 3. 최소 요구되는 연차는 0년차입니다. 4. AI 직무에서의 경력별 채용 특징 및 트렌드 분석:    - 경력이 증가할수록 채용 공고 수는 대체로 감소하는 경향이 있습니다. 이는 AI 분야가 비교적 신규 분야라고 판단되며, 높은 연차의 경력자보다는 초기 단계의 경력자를 더욱 선호하고 있음을 알 수 있습니다.    - 하지만, 예외적으로 3년차에서 가장 많은 채용 공고가 발생하였습니다. 이는 AI 분야에서 3년 정도의 경력을 쌓은 이후에도 적합한 능력이 있으면 채용될 가능성이 높다는 것을 의미하며, 해당 시기가 특정한 능력과 지식을 충분히 갖추게 되는 중요한 시점임을 나타낼 수 있습니다.    - 최고 연차인 10년차의 경우, 공고 수가 매우 적은 것으로 보아 AI 직무에서는 연차가 높을수록 채용 가능성이 낮아지는 경향이 있는 것으로 판단됩니다.

ERP:
1. 가장 많이 요청된 경력 연차는 3년차로 채용 공고 수는 총 11건입니다. 2. 두 번째로 많이 요청된 경력 연차는 4년차로 채용 공고 수는 총 8건입니다. 3. 최소 요구되는 연차는 0년차입니다. 4. ERP 직무에서의 경력별 채용 특징 및 트렌드 분석:    - 주로 3년차 이상의 경력자를 원하는 경향이 있습니다. 특히, 3년차의 공고 수가 가장 많은 것으로 보아 ERP 직무는 일정 수준의 경험이 요구되며, 중급자 위주의 채용이 이루어지는 것으로 해석할 수 있습니다.    - 그러나 0년차를 포함한 신입도 일부 채용되며, 이는 ERP 직무의 특성상 신규 인력도 필요로 한다는 것을 의미할 수 있습니다.    - 전반적으로 이 데이터는 ERP 직무에 대한 경험과 지식이 중요하게 평가되는 것을 보여줍니다.

PM:
1. 가장 많이 요청된 경력 연차는 5년차로, 해당 채용 공고 수는 총 48건입니다. 2. 두 번째로 많이 요청된 경